In [ ]:
# Libraries and definitions for Raman lab

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
nm = 1e-9
cm = 1e-2
c = 299792458 # Speed of light in m/s

def import_image(name):
    """ Import an image and sum over rows. Image must be in same folder as Jupyter notebook. 
    Output:
        x_pixels = numpy list of pixels for the x-axis [0,1,2,...]
        img1d = sum over the rows of img to make a 1D numpy array
        img = the original image as a 2D numpy array"""
    # Import image to numpy array
    img = pd.read_csv(name, sep="\t", header=None)
    img = img.to_numpy() # convert to numpy array
    
    # Sum over rows to make 1D
    img1d = np.sum(img,0)
    x_pixels= np.arange(len(img1d))
    return x_pixels, img1d, img


def peak_finder(data, threshold):
    """ Finds the location of the peaks above a value 'threshold' in a 1D numpy array.
    Output:
        peaks = list indices of found peaks in data"""
    # Find peaks
    peaks, _ = find_peaks(data, height=threshold) # imported from scipy.signal
    
    # Plot spectrum and peaks
    plt.plot(data)
    plt.plot(peaks, data[peaks], "x") # plot cross at each peak
    plt.grid()
    
    # Plot a line at the threshold
    line = np.full_like(data, threshold) 
    plt.plot(line , "--", color="gray")
    
    # Label plot and print peak locations
    plt.xlabel("Pixels")
    plt.ylabel("Counts")
    plt.title("Peak finder")
    print("Peaks found at pixel locations: ", peaks)
    return peaks

def polynomial(x, a, b, c):
    """Polynomial model used for fitting neon peak locations"""
    return a + b*x + c*x**2

def calibrate_pixel_to_wavelength(pixel, peaks, wavelengths):
    """ Convert a list of pixels 'x_pixels' to wavelength 'x_wavelength'.
    Input:
        pixel = list of image pixels [0,1,2,3,...,512]
        peaks = list of the pixels of peaks in the neon image
        wavelengths (nm) = the wavelength of the corresponding peaks in 'peaks'
    Output:
        x_wavelength (nm) = the calibrated x-axis in wavelengths
    """
    # Plot the peak pixels and wavelengths  
    plt.plot(peaks, wavelengths,'.')
    plt.xlabel('Pixels')
    plt.ylabel('Wavelength (nm)')
    
    # Perform curve fit of a second order polynomial
    params, cov = curve_fit(polynomial, peaks, wavelengths, p0=[760,-1,0])
    print("Fitted parameters: ", params)
    plt.plot(peaks, polynomial(peaks, *params)) # * means to insert a list as multiple arguments of a function
    plt.show()
    
    # Use the model to convert pixels to wavelengths
    x_wavelength = polynomial(pixel, *params)
    
    return x_wavelength
